# Imports 

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pyod.models.cblof import CBLOF
from collections import Counter

from itertools import pairwise

In [4]:
!python -m pipreqs.pipreqs

INFO: Successfully saved requirements file in c:\Users\sacho\Documents\5A HEC\strategy\requirements.txt


In [85]:
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from tpot import TPOTClassifier

from sklearn.datasets import load_digits

from sklearn.model_selection import train_test_split

# Preprocessing & feature Engineering

In [2]:
transaction = pd.read_parquet('transaction_data.parquet', engine='pyarrow')

In [3]:
clients = pd.read_csv('sales_client_relationship_dataset.csv')
result = pd.merge(transaction, clients, how="left", on=["client_id", "client_id"])
## we must convert the dates to datetime format
result['date_achat'] = pd.to_datetime(result['date_order'])
result['date_invoice_datetime'] = pd.to_datetime(result['date_invoice'])
result

,date_order,date_invoice,product_id,client_id,sales_net,quantity,order_channel,branch_id,quali_relation,date_achat,date_invoice_datetime
0,2017-09-25,2017-09-25,2376333,188502,155.44320,3,online,2732,Difficult client,2017-09-25,2017-09-25
1,2017-09-25,2017-09-25,2520527,835089,16.39440,3,at the store,10279,Demanding client,2017-09-25,2017-09-25
2,2017-09-25,2017-09-25,2536756,1977896,365.76624,21,online,6184,Agreeable client,2017-09-25,2017-09-25
3,2017-09-25,2017-09-25,3028673,598816,8.59878,201,at the store,4200,Difficult client,2017-09-25,2017-09-25
4,2017-09-25,2017-09-25,203377,2086861,1093.37400,3,by phone,7051,Agreeable client,2017-09-25,2017-09-25
...,...,...,...,...,...,...,...,...,...,...,...
63319310,2019-06-10,2020-06-12,1596284,245272,-2056.91300,5,by phone,7682,Agreeable client,2019-06-10,2020-06-12
63319311,2018-12-14,2020-07-15,908253,880496,27.62760,1001,by phone,9375,Demanding client,2018-12-14,2020-07-15
63319312,2019-07-26,2020-10-29,1168247,2063314,-204.60984,101,by phone,3014,Agreeable client,2019-07-26,2020-10-29
63319313,2018-05-23,2020-11-27,2319965,1287654,70.49730,17,by phone,1015,Difficult client,2018-05-23,2020-11-27


In [18]:
result['order_channel'].describe()

count         63319315
unique               5
top       at the store
freq          32114196
Name: order_channel, dtype: object

In [15]:
result['date_invoice_datetime'].describe()

C:\Users\sacho\AppData\Local\Temp\ipykernel_41520\2754904671.py:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  result['date_invoice_datetime'].describe()


count                63319314
unique                    900
top       2018-08-22 00:00:00
freq                   171453
first     2016-10-12 00:00:00
last      2021-07-13 00:00:00
Name: date_invoice_datetime, dtype: object

In [5]:
add_data = result.groupby('client_id')['date_invoice_datetime'].apply(list)
add_data.to_frame()
add_data1 = pd.DataFrame({'client_id':add_data.index, 'liste_date_achat':add_data.values})
#add_data1

In [12]:
add_data1['nb_orders_2021'].describe()

count    170589.000000
mean          0.003558
std           0.316876
min           0.000000
25%           0.000000
50%           0.000000
75%           0.000000
max          73.000000
Name: nb_orders_2021, dtype: float64

In [9]:
## feature engineering

def function_interval(liste):
    '''
    a function that returns a list for each client of days between commands
    doesn't count multiple commands in one day
    '''
    liste2 = sorted(liste)
    liste3 = [(y - x).days for x, y in pairwise(liste2)]
    liste4 = [i for i in liste3 if i!=0]
    return liste4


#add_data1['nombre_sku_total_commande'] = add_data1['liste_date_achat'].apply(len)
add_data1['nb_total_sku_purchased'] = add_data1['liste_date_achat'].apply(len)
add_data1['interval_list'] = add_data1['liste_date_achat'].apply(function_interval)
#truc['nombre_commande'] = len(truc['interval_list'])
add_data1['frequence_moyenne'] = add_data1['interval_list'].apply(np.mean)
#add_data1[['nombre_commandes'] = add_data1['interval_list'].apply(len)
add_data1['nb_historical_dates'] = add_data1['interval_list'].apply(len)
add_data1

C:\Users\sacho\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\numpy\core\fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,


,client_id,liste_date_achat,nb_total_sku_purchased,interval_list,frequence_moyenne,nb_historical_dates
0,6,"[2017-10-27 00:00:00, 2017-10-27 00:00:00, 201...",40,"[33, 121, 41, 194, 2, 1, 48, 33, 1, 162, 6, 13]",54.583333,12
1,7,"[2018-07-16 00:00:00, 2018-07-16 00:00:00, 201...",15,[161],161.000000,1
2,14,"[2017-09-29 00:00:00, 2017-09-27 00:00:00, 201...",12762,"[1, 1, 1, 1, 3, 1, 1, 1, 1, 3, 1, 1, 1, 1, 3, ...",1.501976,506
3,18,"[2017-09-29 00:00:00, 2017-09-29 00:00:00, 201...",2224,"[1, 2, 3, 8, 1, 1, 1, 3, 1, 6, 1, 3, 4, 1, 2, ...",3.218354,316
4,25,"[2018-10-10 00:00:00, 2018-10-10 00:00:00, 201...",4,[],NaN,0
...,...,...,...,...,...,...
170584,2274456,"[2019-09-02 00:00:00, 2019-09-02 00:00:00, 201...",22,"[2, 1, 8]",3.666667,3
170585,2274479,"[2017-10-30 00:00:00, 2017-10-30 00:00:00, 201...",10,"[1, 484]",242.500000,2
170586,2274504,"[2017-11-02 00:00:00, 2017-11-02 00:00:00, 201...",4,[425],425.000000,1
170587,2274508,"[2017-10-18 00:00:00, 2017-10-18 00:00:00, 201...",8,[132],132.000000,1


In [16]:
#add_data1['nb_historical_dates'] = add_data1['nb_historical_dates'] +1
add_data1['frequence_moyenne'] = add_data1['frequence_moyenne'].fillna(0)
## since the dataset ends at 2021-07-13 00:00:00 (last invoice date), we'll count the number of invoices in the 6 last months [~~ in 2021]

def count_orders_2021(liste):
    '''
    a function that counts the number of invoices in 2021 (last 6 months)
    '''
    liste2 = sorted(liste)
    liste3 = [1 for x in liste2 if (x.year >= 2019)]
    return sum(liste3)


add_data1['nb_last_orders'] = add_data1['liste_date_achat'].apply(count_orders_2021)
add_data1

,client_id,liste_date_achat,nb_total_sku_purchased,interval_list,frequence_moyenne,nb_historical_dates,nb_orders_2021,nb_last_orders
0,6,"[2017-10-27 00:00:00, 2017-10-27 00:00:00, 201...",40,"[33, 121, 41, 194, 2, 1, 48, 33, 1, 162, 6, 13]",54.583333,13,0,17
1,7,"[2018-07-16 00:00:00, 2018-07-16 00:00:00, 201...",15,[161],161.000000,2,0,0
2,14,"[2017-09-29 00:00:00, 2017-09-27 00:00:00, 201...",12762,"[1, 1, 1, 1, 3, 1, 1, 1, 1, 3, 1, 1, 1, 1, 3, ...",1.501976,507,0,5034
3,18,"[2017-09-29 00:00:00, 2017-09-29 00:00:00, 201...",2224,"[1, 2, 3, 8, 1, 1, 1, 3, 1, 6, 1, 3, 4, 1, 2, ...",3.218354,317,5,831
4,25,"[2018-10-10 00:00:00, 2018-10-10 00:00:00, 201...",4,[],0.000000,1,0,0
...,...,...,...,...,...,...,...,...
170584,2274456,"[2019-09-02 00:00:00, 2019-09-02 00:00:00, 201...",22,"[2, 1, 8]",3.666667,4,0,22
170585,2274479,"[2017-10-30 00:00:00, 2017-10-30 00:00:00, 201...",10,"[1, 484]",242.500000,3,0,4
170586,2274504,"[2017-11-02 00:00:00, 2017-11-02 00:00:00, 201...",4,[425],425.000000,2,0,1
170587,2274508,"[2017-10-18 00:00:00, 2017-10-18 00:00:00, 201...",8,[132],132.000000,2,0,0


In [17]:
add_data1.to_csv('table_inter.csv')

In [19]:
add_data_sales = result.groupby('client_id')['sales_net'].apply(sum)

In [31]:
def get_most_common(srs):
    x = list(srs)
    my_counter = Counter(x)
    return my_counter.most_common(1)[0][0]

add_data_relation = result.groupby('client_id')['quali_relation'].agg(get_most_common)
add_data_relation

client_id
6          Agreeable client
7          Difficult client
14         Agreeable client
18         Agreeable client
25         Difficult client
                 ...       
2274456    Difficult client
2274479    Agreeable client
2274504    Demanding client
2274508    Difficult client
2274517    Difficult client
Name: quali_relation, Length: 170589, dtype: object

In [32]:
add_data_channel = result.groupby('client_id')['order_channel'].agg(get_most_common)

In [40]:
#add_data_channel = add_data_channel.to_frame()
#add_data_relation = add_data_relation.to_frame()
#add_data_sales = add_data_sales.to_frame()

In [41]:
add_data2 = pd.merge(add_data1, add_data_sales, how="left", on=["client_id", "client_id"])
add_data3 = pd.merge(add_data2, add_data_relation, how="left", on=["client_id", "client_id"])
add_data4 = pd.merge(add_data3, add_data_channel, how="left", on=["client_id", "client_id"])

In [62]:
dummies = pd.get_dummies(add_data4.quali_relation)
merged = pd.concat([add_data4,dummies],axis = 'columns')

dummies2 = pd.get_dummies(merged.order_channel)
merged2 = pd.concat([merged,dummies2],axis = 'columns')

merged2

,client_id,liste_date_achat,nb_total_sku_purchased,interval_list,frequence_moyenne,nb_historical_dates,nb_orders_2021,nb_last_orders,total_client_sales,relation,...,quali_relation,order_channel,Agreeable client,Demanding client,Difficult client,at the store,by phone,during the visit of a sales rep,online,other
0,6,"[2017-10-27 00:00:00, 2017-10-27 00:00:00, 201...",40,"[33, 121, 41, 194, 2, 1, 48, 33, 1, 162, 6, 13]",54.583333,13,0,17,NaN,NaN,...,Agreeable client,by phone,1,0,0,0,1,0,0,0
1,7,"[2018-07-16 00:00:00, 2018-07-16 00:00:00, 201...",15,[161],161.000000,2,0,0,NaN,NaN,...,Difficult client,at the store,0,0,1,1,0,0,0,0
2,14,"[2017-09-29 00:00:00, 2017-09-27 00:00:00, 201...",12762,"[1, 1, 1, 1, 3, 1, 1, 1, 1, 3, 1, 1, 1, 1, 3, ...",1.501976,507,0,5034,NaN,NaN,...,Agreeable client,by phone,1,0,0,0,1,0,0,0
3,18,"[2017-09-29 00:00:00, 2017-09-29 00:00:00, 201...",2224,"[1, 2, 3, 8, 1, 1, 1, 3, 1, 6, 1, 3, 4, 1, 2, ...",3.218354,317,5,831,NaN,NaN,...,Agreeable client,at the store,1,0,0,1,0,0,0,0
4,25,"[2018-10-10 00:00:00, 2018-10-10 00:00:00, 201...",4,[],0.000000,1,0,0,NaN,NaN,...,Difficult client,at the store,0,0,1,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
170584,2274456,"[2019-09-02 00:00:00, 2019-09-02 00:00:00, 201...",22,"[2, 1, 8]",3.666667,4,0,22,NaN,NaN,...,Difficult client,at the store,0,0,1,1,0,0,0,0
170585,2274479,"[2017-10-30 00:00:00, 2017-10-30 00:00:00, 201...",10,"[1, 484]",242.500000,3,0,4,NaN,NaN,...,Agreeable client,at the store,1,0,0,1,0,0,0,0
170586,2274504,"[2017-11-02 00:00:00, 2017-11-02 00:00:00, 201...",4,[425],425.000000,2,0,1,NaN,NaN,...,Demanding client,at the store,0,1,0,1,0,0,0,0
170587,2274508,"[2017-10-18 00:00:00, 2017-10-18 00:00:00, 201...",8,[132],132.000000,2,0,0,NaN,NaN,...,Difficult client,at the store,0,0,1,1,0,0,0,0


### let's flag the churners 
- clients that haven't purchased something in 2019/2020/2021
- clients that have purchased at least 5 times at ClientCo (independant of the number of items or sku purchased)


In [63]:
merged2.loc[((merged2['nb_historical_dates'] >4) & (merged2['nb_last_orders'] == 0)),'churner'] = 1
merged2['churner'].fillna(0,inplace = True)
merged2['churner'] = merged2['churner'].astype(int)
#merged2

In [64]:
merged2.columns

Index(['client_id', 'liste_date_achat', 'nb_total_sku_purchased',
       'interval_list', 'frequence_moyenne', 'nb_historical_dates',
       'nb_orders_2021', 'nb_last_orders', 'total_client_sales', 'relation',
       'channel', 'sales_net', 'quali_relation', 'order_channel',
       'Agreeable client', 'Demanding client', 'Difficult client',
       'at the store', 'by phone', 'during the visit of a sales rep', 'online',
       'other', 'churner'],
      dtype='object')

In [68]:
data = merged2[['client_id', 'nb_total_sku_purchased', 'frequence_moyenne', 'nb_historical_dates', 'nb_last_orders', 'sales_net',
       'Agreeable client', 'Demanding client', 'Difficult client',
       'at the store', 'by phone', 'during the visit of a sales rep', 'online',
       'other', 'churner']]
data

,client_id,nb_total_sku_purchased,frequence_moyenne,nb_historical_dates,nb_last_orders,sales_net,Agreeable client,Demanding client,Difficult client,at the store,by phone,during the visit of a sales rep,online,other,churner
0,6,40,54.583333,13,17,5.882233e+03,1,0,0,0,1,0,0,0,0
1,7,15,161.000000,2,0,1.190777e+03,0,0,1,1,0,0,0,0,0
2,14,12762,1.501976,507,5034,3.103187e+06,1,0,0,0,1,0,0,0,0
3,18,2224,3.218354,317,831,2.194882e+05,1,0,0,1,0,0,0,0,0
4,25,4,0.000000,1,0,3.745861e+02,0,0,1,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
170584,2274456,22,3.666667,4,22,1.542821e+03,0,0,1,1,0,0,0,0,0
170585,2274479,10,242.500000,3,4,4.186966e+02,1,0,0,1,0,0,0,0,0
170586,2274504,4,425.000000,2,1,1.132566e+02,0,1,0,1,0,0,0,0,0
170587,2274508,8,132.000000,2,0,0.000000e+00,0,0,1,1,0,0,0,0,0


# Build Predictive Classification Model

In [86]:
X = data[['client_id', 'nb_total_sku_purchased', 'frequence_moyenne', 'nb_historical_dates', 'nb_last_orders', 'sales_net',
       'Agreeable client', 'Demanding client', 'Difficult client',
       'at the store', 'by phone', 'during the visit of a sales rep', 'online',
       'other']]

y = data[['churner']]

In [87]:
X.sample(5)

,client_id,nb_total_sku_purchased,frequence_moyenne,nb_historical_dates,nb_last_orders,sales_net,Agreeable client,Demanding client,Difficult client,at the store,by phone,during the visit of a sales rep,online,other
108630,1451246,75,39.333333,16,29,5023.047759,0,0,1,0,0,0,1,0
25747,343429,163,11.728814,60,83,115343.575378,1,0,0,0,1,0,0,0
145821,1945164,165,15.432432,38,25,22965.788318,0,1,0,1,0,0,0,0
158232,2109898,921,3.795122,206,276,89746.814577,1,0,0,1,0,0,0,0
54783,734759,8,174.500000,5,7,809.963584,0,1,0,1,0,0,0,0


In [88]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)

In [89]:
model = XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')

C:\Users\sacho\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\xgboost\sklearn.py:1421: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")


In [90]:
model.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric='mlogloss', feature_types=None, gamma=0, gpu_id=-1,
              grow_policy='depthwise', importance_type=None,
              interaction_constraints='', learning_rate=0.300000012,
              max_bin=256, max_cat_threshold=64, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=100,
              n_jobs=0, num_parallel_tree=1, predictor='auto', random_state=0, ...)

In [91]:
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
accuracy

1.0

AutoML

In [92]:
digits = load_digits()

X_train, X_test, y_train, y_test = train_test_split(digits.data, digits.target, train_size=0.75, test_size=0.25)

pipeline_optimizer = TPOTClassifier(generations=5, population_size=20, cv=5,random_state=42, verbosity=2)
pipeline_optimizer.fit(X_train, y_train)
print(pipeline_optimizer.score(X_test, y_test))

pipeline_optimizer.export('tpot_exported_pipeline.py')

Optimization Progress:   0%|          | 0/120 [00:00<?, ?pipeline/s]


Generation 1 - Current best internal CV score: 0.9829299187663499

Generation 2 - Current best internal CV score: 0.9829299187663499

Generation 3 - Current best internal CV score: 0.9829299187663499

Generation 4 - Current best internal CV score: 0.9829299187663499

Generation 5 - Current best internal CV score: 0.9836734131901419

Best pipeline: KNeighborsClassifier(input_matrix, n_neighbors=2, p=2, weights=distance)
0.9911111111111112


C:\Users\sacho\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_scorer.py:765: FutureWarning: sklearn.metrics.SCORERS is deprecated and will be removed in v1.3. Please use sklearn.metrics.get_scorer_names to get a list of available scorers and sklearn.metrics.get_metric to get scorer.
  warnings.warn(
